In [ ]:
import os, requests
from pathlib import Path
from dotenv import load_dotenv

# notebooks 폴더에서 실행 중이면 루트를 상위로
ROOT = Path.cwd().resolve()
if ROOT.name.lower() == "notebooks":
    ROOT = ROOT.parent

# .env 로드
load_dotenv(ROOT / ".env")

# .env에 아래 중 하나로 저장해두면 됨:
SEOUL_API_KEY = os.getenv("TRAFFIC_KEY")

if not SEOUL_API_KEY:
    raise ValueError("인증키를 못 찾았어. 프로젝트 루트/.env에 SEOUL_OPENAPI_KEY=... 형태로 넣어줘.")

BASE = "http://openapi.seoul.go.kr:8088"

# SpotInfo 1건만 테스트 호출 (정상 응답 : XML)
url = f"{BASE}/{SEOUL_API_KEY}/xml/SpotInfo/1/1/"
r = requests.get(url, timeout=20)

print("status:", r.status_code)
print("url:", url)
print("preview:", r.text[:300])


status: 200
url: http://openapi.seoul.go.kr:8088/5164456e63726f6f36396966455849/xml/SpotInfo/1/1/
preview: <?xml version="1.0" encoding="UTF-8" standalone="yes"?><SpotInfo><list_total_count>139</list_total_count><RESULT><CODE>INFO-000</CODE><MESSAGE>정상 처리되었습니다</MESSAGE></RESULT><row><spot_num>C-02</spot_num><spot_nm>월드컵대교</spot_nm><grs80tm_x>189882</grs80tm_x><grs80tm_y>450789</grs80tm_y></row></SpotInfo


In [2]:
import time
import pandas as pd
import requests
import xml.etree.ElementTree as ET

RAW_DIR = ROOT / "data" / "raw" / "traffic"
RAW_DIR.mkdir(parents=True, exist_ok=True)

OUT_RAW_SPOTS = RAW_DIR / "spotinfo.csv"

def xml_rows(xml_text: str):
    root = ET.fromstring(xml_text)
    rows = root.findall(".//row")
    out = []
    for row in rows:
        d = {}
        for ch in list(row):
            d[ch.tag] = (ch.text.strip() if ch.text else None)
        out.append(d)
    return out

session = requests.Session()

all_rows = []
start = 1
step = 1000

while True:
    url = f"{BASE}/{SEOUL_API_KEY}/xml/SpotInfo/{start}/{start+step-1}/"
    r = session.get(url, timeout=30)
    if r.status_code != 200:
        raise RuntimeError(f"SpotInfo 호출 실패: HTTP {r.status_code}\n{r.text[:300]}\nURL={url}")

    rows = xml_rows(r.text)
    if not rows:
        break

    all_rows.extend(rows)

    # 마지막 페이지면 종료
    if len(rows) < step:
        break

    start += step
    time.sleep(0.2)  # 호출 텀(너무 빠르면 막힐 수 있어서)

spots = pd.DataFrame(all_rows)
spots.columns = [c.lower() for c in spots.columns]

spots.to_csv(OUT_RAW_SPOTS, index=False, encoding="utf-8-sig")

print("✅ saved:", OUT_RAW_SPOTS)
print("shape:", spots.shape)
print("columns:", spots.columns.tolist())
spots.head(5)


✅ saved: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\raw\traffic\spotinfo.csv
shape: (139, 4)
columns: ['spot_num', 'spot_nm', 'grs80tm_x', 'grs80tm_y']


,spot_num,spot_nm,grs80tm_x,grs80tm_y
0,A-01,성산로(금화터널),195489,452136
1,A-02,사직로(사직터널),196756.776106,452546.638644
2,A-03,자하문로(자하문터널),197216.855046,454350.990432
3,A-04,대사관로(삼청터널),198648.893154,455200.108465
4,A-05,율곡로(안국역),198645.671347,452937.216603


In [4]:
import pandas as pd

spots_path = RAW_DIR / "spotinfo.csv"
spots = pd.read_csv(spots_path)
spots.columns = [c.lower() for c in spots.columns]

# 지점번호 컬럼 자동 탐지
spot_id_candidates = [c for c in spots.columns if c in ["spot_num", "spotnum", "spot_no", "spotid"]]
if not spot_id_candidates:
    raise KeyError(f"SpotInfo에서 지점번호 컬럼을 못 찾았어. spots 컬럼: {list(spots.columns)}")

SPOT_COL = spot_id_candidates[0]
spot_list = spots[SPOT_COL].astype(str).dropna().unique().tolist()

print("SPOT_COL:", SPOT_COL)
print("spots count:", len(spot_list))
print("sample:", spot_list[:10])


SPOT_COL: spot_num
spots count: 139
sample: ['A-01', 'A-02', 'A-03', 'A-04', 'A-05', 'A-06', 'A-07', 'A-08', 'A-09', 'A-10']


In [ ]:
import time
import pandas as pd
import requests
import xml.etree.ElementTree as ET

OUT_RAW_HOURLY = RAW_DIR / "volinfo_hourly_raw.csv"

def xml_rows(xml_text: str):
    root = ET.fromstring(xml_text)
    rows = root.findall(".//row")
    out = []
    for row in rows:
        d = {}
        for ch in list(row):
            d[ch.tag] = (ch.text.strip() if ch.text else None)
        out.append(d)
    return out

session = requests.Session()

# 날짜/시간 설정
DATE = "20250115"          # YYYYMMDD
HOURS = ["01", "02", "03"] # 01~02, 02~03, 03~04로 쓸 거

# 혹시 너무 오래 걸리면 N개만 먼저 (문제 없으면 None 유지)
LIMIT_SPOTS = None   # 예: 30 / 아니면 None

spots_to_call = spot_list if LIMIT_SPOTS is None else spot_list[:LIMIT_SPOTS]

records = []
fail = 0

for i, spot in enumerate(spots_to_call, 1):
    for hh in HOURS:
        url = f"{BASE}/{SEOUL_API_KEY}/xml/VolInfo/1/1000/{spot}/{DATE}/{hh}/"
        r = session.get(url, timeout=30)

        if r.status_code != 200:
            fail += 1
            # 너무 로그 길어지지 않게 요약만
            print(f"❌ fail spot={spot} hh={hh} status={r.status_code}")
            continue

        rows = xml_rows(r.text)
        for row in rows:
            row2 = {k.lower(): v for k, v in row.items()}
            row2["spot"] = str(spot)
            row2["yyyymmdd"] = DATE
            row2["hh"] = hh
            records.append(row2)

    if i % 10 == 0:
        print(f"progress: {i}/{len(spots_to_call)} (records={len(records)}, fail={fail})")
        time.sleep(0.2)  # 호출 텀

raw = pd.DataFrame(records)
raw.to_csv(OUT_RAW_HOURLY, index=False, encoding="utf-8-sig")

print("✅ saved:", OUT_RAW_HOURLY)
print("raw shape:", raw.shape)
print("fail count:", fail)
raw.head(5)


progress: 10/139 (records=153, fail=0)
progress: 20/139 (records=306, fail=0)
progress: 30/139 (records=420, fail=0)
progress: 40/139 (records=699, fail=0)
progress: 50/139 (records=837, fail=0)
progress: 60/139 (records=1053, fail=0)
progress: 70/139 (records=1239, fail=0)
progress: 80/139 (records=1437, fail=0)
progress: 90/139 (records=1593, fail=0)
progress: 100/139 (records=1782, fail=0)
progress: 110/139 (records=1959, fail=0)
progress: 120/139 (records=2169, fail=0)
progress: 130/139 (records=2397, fail=0)
✅ saved: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\raw\traffic\volinfo_hourly_raw.csv
raw shape: (2553, 8)
fail count: 0


,spot_num,ymd,hh,io_type,lane_num,vol,spot,yyyymmdd
0,A-01,20250115,01,1,1,165,A-01,20250115
1,A-01,20250115,01,1,2,142,A-01,20250115
2,A-01,20250115,01,2,1,261,A-01,20250115
3,A-01,20250115,01,2,2,169,A-01,20250115
4,A-01,20250115,02,1,1,87,A-01,20250115


In [ ]:
import pandas as pd

RAW_HOURLY = RAW_DIR / "volinfo_hourly_raw.csv"
raw = pd.read_csv(RAW_HOURLY)
raw.columns = [c.lower() for c in raw.columns]

# 교통량 값 컬럼 자동 탐지
vol_candidates = [c for c in raw.columns if c in ["vol", "volume", "traffic", "trfvlm", "traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼을 못 찾았어. raw 컬럼: {list(raw.columns)}")
VOL_COL = vol_candidates[0]

# 타입 정리
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)
raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)

# 같은 spot+hh에 여러 row가 있을 수 있어서 안전하게 합으로 정리
spot_hh = (
    raw.groupby(["spot", "yyyymmdd", "hh"], as_index=False)[VOL_COL]
       .sum()
)

# 피벗: spot 행, hh 열
wide = spot_hh.pivot(index=["spot","yyyymmdd"], columns="hh", values=VOL_COL).fillna(0).reset_index()

# 컬럼명
rename_map = {
    "01": "traffic_01_02",
    "02": "traffic_02_03",
    "03": "traffic_03_04"
}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh: newc}, inplace=True)
    else:
        wide[newc] = 0

OUT_SPOT_WIDE = RAW_DIR / "volinfo_spot_traffic_3cols.csv"
wide.to_csv(OUT_SPOT_WIDE, index=False, encoding="utf-8-sig")

print("✅ VOL_COL:", VOL_COL)
print("✅ saved:", OUT_SPOT_WIDE)
print("wide shape:", wide.shape)
wide.head(10)


✅ VOL_COL: vol
✅ saved: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\raw\traffic\volinfo_spot_traffic_3cols.csv
wide shape: (130, 5)


hh,spot,yyyymmdd,traffic_01_02,traffic_02_03,traffic_03_04
0,A-01,20250115,737,566,465
1,A-02,20250115,721,535,469
2,A-03,20250115,245,157,133
3,A-04,20250115,0,1,0
4,A-05,20250115,858,636,451
5,A-06,20250115,768,572,467
6,A-07,20250115,576,417,355
7,A-08,20250115,1196,969,865
8,A-09,20250115,982,728,681
9,A-10,20250115,636,435,435


In [7]:
import geopandas as gpd
from pathlib import Path

PRO_DIR = ROOT / "data" / "processed"

# processed 안 geojson 후보 싹 찾기
cands = sorted(PRO_DIR.glob("*.geojson"))

print("geojson candidates:", len(cands))
for p in cands[:30]:
    print(" -", p.name)

# 성수/250/grid 키워드로 우선순위 탐색
def score(name: str):
    n = name.lower()
    s = 0
    if "seongsu" in n: s += 5
    if "sungsu" in n:  s += 4
    if "250" in n:     s += 4
    if "grid" in n:    s += 3
    return s

best = None
best_score = -1
for p in cands:
    sc = score(p.name)
    if sc > best_score:
        best_score = sc
        best = p

print("\nbest guess:", best, "score:", best_score)

if best is None or best_score <= 0:
    raise FileNotFoundError(
        "processed 폴더에서 성수 250m 격자 geojson 후보를 못 찾았어. "
        "seongsu_grid_250m.geojson(또는 비슷한 이름) 파일을 data/processed에 넣어줘."
    )

grid = gpd.read_file(best)
grid.columns = [c.lower() for c in grid.columns]

print("✅ loaded:", best.name)
print("shape:", grid.shape)
print("columns:", list(grid.columns))

# grid_id 유무 체크
if "grid_id" not in grid.columns:
    raise KeyError(f"격자 파일에 grid_id 컬럼이 없어. 현재 컬럼: {list(grid.columns)}")

grid[["grid_id"]].head(5)


geojson candidates: 1
 - seongsu_grid_250m_enriched.geojson

best guess: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\processed\seongsu_grid_250m_enriched.geojson score: 12
✅ loaded: seongsu_grid_250m_enriched.geojson
shape: (109, 5)
columns: ['grid_id', 'streetlight_cnt', 'cctv_cnt', 'park_cnt', 'geometry']


,grid_id
0,0
1,1
2,2
3,3
4,4


In [ ]:
import time
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
import geopandas as gpd

# 기간/시간대
DATES = pd.date_range("2025-12-01", "2025-12-14", freq="D").strftime("%Y%m%d").tolist()
HOURS = ["01", "02", "03"]  # 01~02, 02~03, 03~04

OUT_RAW_MULTI = RAW_DIR / "volinfo_hourly_raw_20251201_1214.csv"
OUT_CSV = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"
OUT_GEOJSON = PRO_DIR / "seongsu_grid_250m_plus_traffic_3cols_median_20251201_1214.geojson"

print("DATES:", DATES[0], "~", DATES[-1], "| days:", len(DATES))
print("spots:", len(spot_list))

# XML -> rows 파서
def xml_rows(xml_text: str):
    root = ET.fromstring(xml_text)
    rows = root.findall(".//row")
    out = []
    for row in rows:
        d = {}
        for ch in list(row):
            d[ch.tag] = (ch.text.strip() if ch.text else None)
        out.append(d)
    return out

# VolInfo 다중 날짜 수집 (raw)
session = requests.Session()

records = []
fail = 0
total_calls = len(spot_list) * len(DATES) * len(HOURS)
done = 0

for i, spot in enumerate(spot_list, 1):
    spot = str(spot)
    for d in DATES:
        for hh in HOURS:
            url = f"{BASE}/{SEOUL_API_KEY}/xml/VolInfo/1/1000/{spot}/{d}/{hh}/"
            r = session.get(url, timeout=30)
            done += 1

            if r.status_code != 200:
                fail += 1
                if fail <= 10:
                    print(f"❌ fail spot={spot} date={d} hh={hh} status={r.status_code}")
                continue

            rows = xml_rows(r.text)
            for row in rows:
                row2 = {k.lower(): v for k, v in row.items()}
                row2["spot"] = spot
                row2["yyyymmdd"] = d
                row2["hh"] = hh
                records.append(row2)

        # 너무 빠르면 막힐 수 있어서 살짝 텀
        time.sleep(0.05)

    if i % 10 == 0:
        print(f"progress: {i}/{len(spot_list)} spots | calls {done}/{total_calls} | records={len(records)} | fail={fail}")

raw = pd.DataFrame(records)
raw.to_csv(OUT_RAW_MULTI, index=False, encoding="utf-8-sig")
print("✅ saved raw:", OUT_RAW_MULTI, "shape:", raw.shape, "fail:", fail)

if raw.empty:
    raise ValueError("raw가 비었어. (키/날짜/spot 형식/호출 제한) 중 하나 문제일 가능성 큼.")

# raw -> spot-day-hh로 합산 -> spot-day wide(3칼럼)
raw.columns = [c.lower() for c in raw.columns]

vol_candidates = [c for c in raw.columns if c in ["vol", "volume", "traffic", "trfvlm", "traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼을 못 찾았어. raw 컬럼: {list(raw.columns)}")
VOL_COL = vol_candidates[0]
print("VOL_COL:", VOL_COL)

raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)

spot_day_hh = (
    raw.groupby(["spot", "yyyymmdd", "hh"], as_index=False)[VOL_COL]
       .sum()
)

wide = (
    spot_day_hh.pivot(index=["spot", "yyyymmdd"], columns="hh", values=VOL_COL)
              .fillna(0)
              .reset_index()
)

rename_map = {"01": "traffic_01_02", "02": "traffic_02_03", "03": "traffic_03_04"}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh: newc}, inplace=True)
    else:
        wide[newc] = 0

wide = wide[["spot","yyyymmdd","traffic_01_02","traffic_02_03","traffic_03_04"]].copy()
print("wide spot-day shape:", wide.shape)
wide.head()

# SpotInfo(TM좌표) -> grid_id 매핑 만들기 (5186/5181 자동 시도)
spots = pd.read_csv(RAW_DIR / "spotinfo.csv")
spots.columns = [c.lower() for c in spots.columns]

# 데이터
SPOT_COL = "spot_num"
X_COL = "grs80tm_x"
Y_COL = "grs80tm_y"

spots[SPOT_COL] = spots[SPOT_COL].astype(str)
spots[X_COL] = pd.to_numeric(spots[X_COL], errors="coerce")
spots[Y_COL] = pd.to_numeric(spots[Y_COL], errors="coerce")
spots = spots.dropna(subset=[X_COL, Y_COL]).copy()

if grid.crs is None:
    grid = grid.set_crs("EPSG:4326", allow_override=True)

candidates = ["EPSG:5186", "EPSG:5181"]
best_joined, best_n, best_epsg = None, -1, None

for epsg in candidates:
    g_spots = gpd.GeoDataFrame(
        spots[[SPOT_COL, X_COL, Y_COL]].copy(),
        geometry=gpd.points_from_xy(spots[X_COL], spots[Y_COL]),
        crs=epsg
    ).to_crs(grid.crs)

    joined = gpd.sjoin(g_spots, grid[["grid_id", "geometry"]], how="inner", predicate="within")
    if len(joined) > best_n:
        best_joined, best_n, best_epsg = joined, len(joined), epsg

print("✅ best spot CRS:", best_epsg, "| joined rows:", best_n)

spot_to_grid = best_joined[[SPOT_COL, "grid_id"]].drop_duplicates().copy()
spot_to_grid.rename(columns={SPOT_COL: "spot"}, inplace=True)
spot_to_grid["spot"] = spot_to_grid["spot"].astype(str)

# spot-day wide + grid_id 붙여서 grid-day 합산
tmp = wide.merge(spot_to_grid, on="spot", how="left")
print("unmapped rows:", tmp["grid_id"].isna().sum())
tmp = tmp.dropna(subset=["grid_id"]).copy()

for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    tmp[c] = pd.to_numeric(tmp[c], errors="coerce").fillna(0)

grid_day = (
    tmp.groupby(["grid_id","yyyymmdd"], as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]]
       .sum()
)
print("grid-day shape:", grid_day.shape)

# 14일 중앙값(median)으로 grid_id 대표값 만들기
grid_med = (
    grid_day.groupby("grid_id", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]]
            .median()
)

grid_med.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("✅ saved csv:", OUT_CSV, "shape:", grid_med.shape)

# 격자 geojson에도 붙여서 저장
grid2 = grid.merge(grid_med, on="grid_id", how="left")
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    grid2[c] = grid2[c].fillna(0)

grid2.to_file(OUT_GEOJSON, driver="GeoJSON", encoding="utf-8")
print("✅ saved geojson:", OUT_GEOJSON)

grid2[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]].head(10)


DATES: 20251201 ~ 20251214 | days: 14
spots: 139
progress: 10/139 spots | calls 420/5838 | records=1848 | fail=0
progress: 20/139 spots | calls 840/5838 | records=4410 | fail=0
progress: 30/139 spots | calls 1260/5838 | records=6333 | fail=0
progress: 40/139 spots | calls 1680/5838 | records=9747 | fail=0
progress: 50/139 spots | calls 2100/5838 | records=12267 | fail=0
progress: 60/139 spots | calls 2520/5838 | records=15174 | fail=0


ConnectionError: ('Connection aborted.', ConnectionResetError(10054, '현재 연결은 원격 호스트에 의해 강제로 끊겼습니다', None, 10054, None))

In [ ]:
import time, random
import pandas as pd
import numpy as np
import requests
import xml.etree.ElementTree as ET
from pathlib import Path

DATES = pd.date_range("2025-12-01", "2025-12-14", freq="D").strftime("%Y%m%d").tolist()
HOURS = ["01", "02", "03"]

OUT_RAW_MULTI = RAW_DIR / "volinfo_hourly_raw_20251201_1214.csv"
OUT_CSV = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"
OUT_GEOJSON = PRO_DIR / "seongsu_grid_250m_plus_traffic_3cols_median_20251201_1214.geojson"

# XML parser
def xml_rows(xml_text: str):
    if not xml_text or not str(xml_text).strip():
        return []
    txt = str(xml_text).strip()
    # 에러메시지/HTML/빈값이 섞여 들어오는 경우 방어
    if "<row" not in txt:
        return []
    try:
        root = ET.fromstring(txt)
    except ET.ParseError:
        return []
    rows = root.findall(".//row")
    out = []
    for row in rows:
        d = {}
        for ch in list(row):
            d[ch.tag] = (ch.text.strip() if ch.text else None)
        out.append(d)
    return out


# safe fetch with retry/backoff
session = requests.Session()

def safe_get(url, timeout=30, max_retry=10, base_sleep=0.7):
    last = None
    for k in range(max_retry):
        try:
            r = session.get(url, timeout=timeout)
            if r.status_code == 200:
                txt = (r.text or "").strip()
                # XML이 너무 짧거나 row가 아예 없으면 서버가 헛응답 준 걸로 보고 재시도
                if len(txt) < 50:
                    raise RuntimeError("empty/too short body")
                return r
            last = RuntimeError(f"HTTP {r.status_code}")
        except Exception as e:
            last = e
        time.sleep(base_sleep * (2 ** min(k, 4)) + random.uniform(0, 0.4))
    raise last
    if r.status_code == 200:
        txt = (r.text or "").strip()
    if "<row" not in txt:
        raise RuntimeError("no <row> in body")
    return r



# resume: already collected keys
done_keys = set()
if OUT_RAW_MULTI.exists():
    prev = pd.read_csv(OUT_RAW_MULTI, usecols=["spot","yyyymmdd","hh"])
    prev["spot"] = prev["spot"].astype(str)
    prev["yyyymmdd"] = prev["yyyymmdd"].astype(str)
    prev["hh"] = prev["hh"].astype(str).str.zfill(2)
    done_keys = set(zip(prev["spot"], prev["yyyymmdd"], prev["hh"]))
    print("✅ resume from existing raw:", OUT_RAW_MULTI, "| done keys:", len(done_keys))
else:
    print("✅ start fresh (no existing raw)")

# main loop
records = []
fail = 0
saved = 0

total_calls = len(spot_list) * len(DATES) * len(HOURS)
done = 0

for i, spot in enumerate(spot_list, 1):
    spot = str(spot)
    for d in DATES:
        for hh in HOURS:
            done += 1
            key = (spot, d, hh)
            if key in done_keys:
                continue

            url = f"{BASE}/{SEOUL_API_KEY}/xml/VolInfo/1/1000/{spot}/{d}/{hh}/"

            try:
                r = safe_get(url, timeout=30, max_retry=8, base_sleep=0.6)
            except Exception:
                fail += 1
                # 실패가 누적되면 잠깐 쉬어주기
                if fail % 10 == 0:
                    time.sleep(10)
                continue

            rows = xml_rows(r.text)
            for row in rows:
                row2 = {k.lower(): v for k, v in row.items()}
                row2["spot"] = spot
                row2["yyyymmdd"] = d
                row2["hh"] = hh
                records.append(row2)

            # 요청 간 텀 (너무 빠르면 다시 끊김)
            time.sleep(0.12 + random.uniform(0, 0.08))

            # ---- flush: 5000행마다 raw에 append 저장 ----
            if len(records) >= 5000:
                df = pd.DataFrame(records)
                mode = "a" if OUT_RAW_MULTI.exists() else "w"
                header = not OUT_RAW_MULTI.exists()
                df.to_csv(OUT_RAW_MULTI, mode=mode, header=header, index=False, encoding="utf-8-sig")
                saved += len(records)
                records = []
                print(f"💾 flushed rows: total_saved≈{saved} | fail={fail} | progress calls={done}/{total_calls}")

    if i % 10 == 0:
        print(f"progress: {i}/{len(spot_list)} spots | fail={fail} | calls={done}/{total_calls}")

# 마지막 남은 records flush
if records:
    df = pd.DataFrame(records)
    mode = "a" if OUT_RAW_MULTI.exists() else "w"
    header = not OUT_RAW_MULTI.exists()
    df.to_csv(OUT_RAW_MULTI, mode=mode, header=header, index=False, encoding="utf-8-sig")
    saved += len(records)

print("✅ raw saved:", OUT_RAW_MULTI, "| added rows:", saved, "| fail:", fail)


# 이후는 raw -> grid median (이전 셀의 아래 부분 그대로)
raw = pd.read_csv(OUT_RAW_MULTI)
raw.columns = [c.lower() for c in raw.columns]

vol_candidates = [c for c in raw.columns if c in ["vol", "volume", "traffic", "trfvlm", "traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼을 못 찾았어. raw 컬럼: {list(raw.columns)}")
VOL_COL = vol_candidates[0]
print("VOL_COL:", VOL_COL)

raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)

spot_day_hh = raw.groupby(["spot","yyyymmdd","hh"], as_index=False)[VOL_COL].sum()
wide = spot_day_hh.pivot(index=["spot","yyyymmdd"], columns="hh", values=VOL_COL).fillna(0).reset_index()

rename_map = {"01": "traffic_01_02", "02": "traffic_02_03", "03": "traffic_03_04"}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh: newc}, inplace=True)
    else:
        wide[newc] = 0

wide = wide[["spot","yyyymmdd","traffic_01_02","traffic_02_03","traffic_03_04"]].copy()

# spot->grid 매핑 (TM좌표 5186/5181 자동)
import geopandas as gpd

spots = pd.read_csv(RAW_DIR / "spotinfo.csv")
spots.columns = [c.lower() for c in spots.columns]
SPOT_COL = "spot_num"
X_COL = "grs80tm_x"
Y_COL = "grs80tm_y"

spots[SPOT_COL] = spots[SPOT_COL].astype(str)
spots[X_COL] = pd.to_numeric(spots[X_COL], errors="coerce")
spots[Y_COL] = pd.to_numeric(spots[Y_COL], errors="coerce")
spots = spots.dropna(subset=[X_COL, Y_COL]).copy()

if grid.crs is None:
    grid = grid.set_crs("EPSG:4326", allow_override=True)

best_joined, best_n, best_epsg = None, -1, None
for epsg in ["EPSG:5186","EPSG:5181"]:
    g_spots = gpd.GeoDataFrame(
        spots[[SPOT_COL, X_COL, Y_COL]].copy(),
        geometry=gpd.points_from_xy(spots[X_COL], spots[Y_COL]),
        crs=epsg
    ).to_crs(grid.crs)
    joined = gpd.sjoin(g_spots, grid[["grid_id","geometry"]], how="inner", predicate="within")
    if len(joined) > best_n:
        best_joined, best_n, best_epsg = joined, len(joined), epsg

print("✅ best spot CRS:", best_epsg, "| joined rows:", best_n)

spot_to_grid = best_joined[[SPOT_COL,"grid_id"]].drop_duplicates().copy()
spot_to_grid.rename(columns={SPOT_COL:"spot"}, inplace=True)
spot_to_grid["spot"] = spot_to_grid["spot"].astype(str)

tmp = wide.merge(spot_to_grid, on="spot", how="left").dropna(subset=["grid_id"]).copy()
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    tmp[c] = pd.to_numeric(tmp[c], errors="coerce").fillna(0)

grid_day = tmp.groupby(["grid_id","yyyymmdd"], as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum()
grid_med = grid_day.groupby("grid_id", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].median()

grid_med.to_csv(OUT_CSV, index=False, encoding="utf-8-sig")
print("✅ saved csv:", OUT_CSV, "shape:", grid_med.shape)

grid2 = grid.merge(grid_med, on="grid_id", how="left")
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    grid2[c] = grid2[c].fillna(0)

grid2.to_file(OUT_GEOJSON, driver="GeoJSON", encoding="utf-8")
print("✅ saved geojson:", OUT_GEOJSON)

grid2[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]].head(10)


✅ resume from existing raw: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\raw\traffic\volinfo_hourly_raw_20251201_1214.csv | done keys: 3242
progress: 10/139 spots | fail=0 | calls=420/5838
progress: 20/139 spots | fail=0 | calls=840/5838
progress: 30/139 spots | fail=0 | calls=1260/5838
progress: 40/139 spots | fail=0 | calls=1680/5838
progress: 50/139 spots | fail=0 | calls=2100/5838
progress: 60/139 spots | fail=0 | calls=2520/5838
progress: 70/139 spots | fail=0 | calls=2940/5838
progress: 80/139 spots | fail=0 | calls=3360/5838
progress: 90/139 spots | fail=0 | calls=3780/5838
💾 flushed rows: total_saved≈5007 | fail=0 | progress calls=4139/5838
progress: 100/139 spots | fail=0 | calls=4200/5838
progress: 110/139 spots | fail=0 | calls=4620/5838
💾 flushed rows: total_saved≈10007 | fail=0 | progress calls=4862/5838
progress: 120/139 spots | fail=0 | calls=5040/5838
progress: 130/139 spots | fail=0 | calls=5460/5838
💾 flushed rows: total_saved≈15007 | fail=0 | progress calls=54

,grid_id,traffic_01_02,traffic_02_03,traffic_03_04
0,0,0.0,0.0,0.0
1,1,0.0,0.0,0.0
2,2,0.0,0.0,0.0
3,3,0.0,0.0,0.0
4,4,0.0,0.0,0.0
5,5,0.0,0.0,0.0
6,6,0.0,0.0,0.0
7,7,0.0,0.0,0.0
8,8,0.0,0.0,0.0
9,9,0.0,0.0,0.0


In [ ]:
import pandas as pd
from pathlib import Path

PRO_DIR = ROOT / "data" / "processed"

BASE_FEATURES = PRO_DIR / "grid_features_base.csv"
TRAFFIC_3COLS = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"
OUT_FINAL     = PRO_DIR / "grid_features_base_plus_traffic3cols_20251201_1214.csv"

base = pd.read_csv(BASE_FEATURES, dtype={"grid_id": str})
traffic = pd.read_csv(TRAFFIC_3COLS, dtype={"grid_id": str})

# grid_id 컬럼 대소문자 통일
base.columns = [c if c.lower() != "grid_id" else "grid_id" for c in base.columns]
traffic.columns = [c if c.lower() != "grid_id" else "grid_id" for c in traffic.columns]

# 중복 방어(거의 없겠지만 안전하게)
traffic = traffic.groupby("grid_id", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].mean()

merged = base.merge(traffic, on="grid_id", how="left")

for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    merged[c] = merged[c].fillna(0)

merged.to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")

print("✅ base:", base.shape)
print("✅ traffic:", traffic.shape)
print("✅ merged:", merged.shape)
print("✅ saved:", OUT_FINAL)

merged[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]].head(10)


✅ base: (109, 3)
✅ traffic: (2, 4)
✅ merged: (109, 6)
✅ saved: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\processed\grid_features_base_plus_traffic3cols_20251201_1214.csv


,grid_id,traffic_01_02,traffic_02_03,traffic_03_04
0,0,0.0,0.0,0.0
1,1,0.0,0.0,0.0
2,2,0.0,0.0,0.0
3,3,0.0,0.0,0.0
4,4,0.0,0.0,0.0
5,5,0.0,0.0,0.0
6,6,0.0,0.0,0.0
7,7,0.0,0.0,0.0
8,8,0.0,0.0,0.0
9,9,0.0,0.0,0.0


In [15]:
import pandas as pd
from pathlib import Path

PRO_DIR = ROOT / "data" / "processed"

BASE_FEATURES = PRO_DIR / "grid_features_base.csv"
TRAFFIC_3COLS = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"

base = pd.read_csv(BASE_FEATURES, dtype={"grid_id": str})
traffic = pd.read_csv(TRAFFIC_3COLS, dtype={"grid_id": str})

# 컬럼 통일
base.columns = [c if c.lower() != "grid_id" else "grid_id" for c in base.columns]
traffic.columns = [c if c.lower() != "grid_id" else "grid_id" for c in traffic.columns]

# 값이 진짜 0뿐인지 확인
print("traffic head:\n", traffic.head())
print("\ntraffic stats (sum):")
print(traffic[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum())

nonzero_rows = (traffic[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum(axis=1) > 0).sum()
print("\ntraffic nonzero rows:", nonzero_rows, "/", len(traffic))

# grid_id 샘플 비교
print("\nbase grid_id sample:", base["grid_id"].astype(str).head(5).tolist())
print("traffic grid_id sample:", traffic["grid_id"].astype(str).head(5).tolist())

# 교집합 개수 확인(merge가 되는지)
base_ids = set(base["grid_id"].astype(str))
traffic_ids = set(traffic["grid_id"].astype(str))
print("\nintersection count:", len(base_ids & traffic_ids))


traffic head:
   grid_id  traffic_01_02  traffic_02_03  traffic_03_04
0    10.0         2808.0         2278.0         1958.0
1    85.0         2361.0         1926.0         1490.0

traffic stats (sum):
traffic_01_02    5169.0
traffic_02_03    4204.0
traffic_03_04    3448.0
dtype: float64

traffic nonzero rows: 2 / 2

base grid_id sample: ['0', '1', '2', '3', '4']
traffic grid_id sample: ['10.0', '85.0']

intersection count: 0


In [16]:
import re

def clean_gid(s):
    s = str(s).strip()
    s = re.sub(r"\.0$", "", s)   # 123.0 -> 123
    return s

base["grid_id"] = base["grid_id"].apply(clean_gid)
traffic["grid_id"] = traffic["grid_id"].apply(clean_gid)

print("intersection after clean:", len(set(base["grid_id"]) & set(traffic["grid_id"])))


intersection after clean: 2


In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

PRO_DIR = ROOT / "data" / "processed"
RAW_DIR = ROOT / "data" / "raw" / "traffic"

GRID_PATH = PRO_DIR / "seongsu_grid_250m_enriched.geojson"
RAW_MULTI = RAW_DIR / "volinfo_hourly_raw_20251201_1214.csv"
SPOTINFO  = RAW_DIR / "spotinfo.csv"

OUT_TRAFFIC = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"

grid = gpd.read_file(GRID_PATH)
grid.columns = [c.lower() for c in grid.columns]
if grid.crs is None:
    grid = grid.set_crs("EPSG:4326", allow_override=True)

raw = pd.read_csv(RAW_MULTI)
raw.columns = [c.lower() for c in raw.columns]

# 교통량 값 컬럼 자동탐지
vol_candidates = [c for c in raw.columns if c in ["vol","volume","traffic","trfvlm","traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼 못 찾음. raw cols={list(raw.columns)}")
VOL_COL = vol_candidates[0]

raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)

# spot-day-hh 합산 -> spot-day wide(3칼럼)
spot_day_hh = raw.groupby(["spot","yyyymmdd","hh"], as_index=False)[VOL_COL].sum()
wide = spot_day_hh.pivot(index=["spot","yyyymmdd"], columns="hh", values=VOL_COL).fillna(0).reset_index()

rename_map = {"01":"traffic_01_02","02":"traffic_02_03","03":"traffic_03_04"}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh:newc}, inplace=True)
    else:
        wide[newc] = 0
wide = wide[["spot","yyyymmdd","traffic_01_02","traffic_02_03","traffic_03_04"]].copy()

# SpotInfo (TM)
spots = pd.read_csv(SPOTINFO)
spots.columns = [c.lower() for c in spots.columns]
SPOT_COL = "spot_num"
X_COL = "grs80tm_x"
Y_COL = "grs80tm_y"

spots[SPOT_COL] = spots[SPOT_COL].astype(str)
spots[X_COL] = pd.to_numeric(spots[X_COL], errors="coerce")
spots[Y_COL] = pd.to_numeric(spots[Y_COL], errors="coerce")
spots = spots.dropna(subset=[X_COL, Y_COL]).copy()

# EPSG 후보를 넓게 돌려서 "가장 많이 매핑되는" 좌표계를 자동 선택
epsg_candidates = ["EPSG:5186","EPSG:5181","EPSG:5179","EPSG:5187","EPSG:5183","EPSG:5174","EPSG:5178"]
best = {"epsg": None, "n": -1, "joined": None}

for epsg in epsg_candidates:
    g_spots = gpd.GeoDataFrame(
        spots[[SPOT_COL, X_COL, Y_COL]].copy(),
        geometry=gpd.points_from_xy(spots[X_COL], spots[Y_COL]),
        crs=epsg
    ).to_crs(grid.crs)

    joined = gpd.sjoin(g_spots, grid[["grid_id","geometry"]], how="inner", predicate="within")
    n = len(joined)
    print(epsg, "joined_rows=", n)
    if n > best["n"]:
        best = {"epsg": epsg, "n": n, "joined": joined}

print("\n✅ BEST EPSG:", best["epsg"], "| joined_rows:", best["n"])
if best["n"] <= 10:
    raise RuntimeError("매핑이 너무 적게 됐어. 격자 CRS/spot 좌표가 서로 안 맞을 가능성이 큼. joined_rows가 최소 수백은 나와야 정상.")

spot_to_grid = best["joined"][[SPOT_COL,"grid_id"]].drop_duplicates().copy()
spot_to_grid.rename(columns={SPOT_COL:"spot"}, inplace=True)
spot_to_grid["spot"] = spot_to_grid["spot"].astype(str)

tmp = wide.merge(spot_to_grid, on="spot", how="inner")

grid_day = tmp.groupby(["grid_id","yyyymmdd"], as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum()
grid_med = grid_day.groupby("grid_id", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].median()

grid_med.to_csv(OUT_TRAFFIC, index=False, encoding="utf-8-sig")

print("\n✅ saved:", OUT_TRAFFIC)
print("grid_med rows:", len(grid_med))
print("nonzero grids:", (grid_med[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum(axis=1) > 0).sum())
grid_med.head()


EPSG:5186 joined_rows= 0
EPSG:5181 joined_rows= 2
EPSG:5179 joined_rows= 0
EPSG:5187 joined_rows= 0
EPSG:5183 joined_rows= 0
EPSG:5174 joined_rows= 2
EPSG:5178 joined_rows= 0

✅ BEST EPSG: EPSG:5181 | joined_rows: 2


RuntimeError: 매핑이 너무 적게 됐어. 격자 CRS/spot 좌표가 서로 안 맞을 가능성이 큼. joined_rows가 최소 수백은 나와야 정상.

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd

PRO_DIR = ROOT / "data" / "processed"
RAW_DIR = ROOT / "data" / "raw" / "traffic"

GRID_PATH = PRO_DIR / "seongsu_grid_250m_enriched.geojson"
RAW_MULTI = RAW_DIR / "volinfo_hourly_raw_20251201_1214.csv"
SPOTINFO  = RAW_DIR / "spotinfo.csv"

OUT_TRAFFIC = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214.csv"

# 로드 (절대 grid CRS를 임의로 set 하지 말고 원본 그대로 할 것)
grid0 = gpd.read_file(GRID_PATH)
grid0.columns = [c.lower() for c in grid0.columns]
if "grid_id" not in grid0.columns:
    raise KeyError(f"grid_id 없음. grid cols={list(grid0.columns)}")

spots = pd.read_csv(SPOTINFO)
spots.columns = [c.lower() for c in spots.columns]
SPOT_COL = "spot_num"
X_COL = "grs80tm_x"
Y_COL = "grs80tm_y"

spots[SPOT_COL] = spots[SPOT_COL].astype(str)
spots[X_COL] = pd.to_numeric(spots[X_COL], errors="coerce")
spots[Y_COL] = pd.to_numeric(spots[Y_COL], errors="coerce")
spots = spots.dropna(subset=[X_COL, Y_COL]).copy()

print("grid.crs(original):", grid0.crs)
print("grid bounds:", grid0.total_bounds)  # [minx, miny, maxx, maxy]
print("spot x/y range:", (spots[X_COL].min(), spots[X_COL].max()), (spots[Y_COL].min(), spots[Y_COL].max()))

# 후보 CRS 세트
# grid CRS가 파일에 있으면 그걸 우선 사용, 없으면 후보를 넓게 탐색
if grid0.crs is not None:
    grid_crs_candidates = [grid0.crs.to_string()]
else:
    # geojson인데도 CRS 누락되는 경우 많아서 광범위 후보
    grid_crs_candidates = ["EPSG:4326", "EPSG:5179", "EPSG:5186", "EPSG:5181", "EPSG:5187", "EPSG:5183", "EPSG:3857"]

# SpotInfo 좌표는 TM 계열일 확률 높아서 후보 여러 개
spot_crs_candidates = ["EPSG:5186","EPSG:5181","EPSG:5179","EPSG:5187","EPSG:5183","EPSG:5174","EPSG:5178","EPSG:3857","EPSG:4326"]

best = {"grid_crs": None, "spot_crs": None, "joined": None, "n_rows": -1, "n_spots": -1}

# grid CRS × spot CRS 조합 탐색
for gcrs in grid_crs_candidates:
    grid = grid0.copy()
    # grid 좌표 자체를 "변환"하면 망할 수 있어서: CRS만 가정해서 박아넣고(allow_override), 좌표는 그대로 둠
    grid = grid.set_crs(gcrs, allow_override=True)

    for scrs in spot_crs_candidates:
        g_spots = gpd.GeoDataFrame(
            spots[[SPOT_COL, X_COL, Y_COL]].copy(),
            geometry=gpd.points_from_xy(spots[X_COL], spots[Y_COL]),
            crs=scrs
        )
        try:
            g_spots2 = g_spots.to_crs(gcrs)
        except Exception:
            continue

        try:
            joined = gpd.sjoin(g_spots2, grid[["grid_id","geometry"]], how="inner", predicate="within")
        except Exception:
            continue

        n_rows = len(joined)
        n_spots = joined[SPOT_COL].nunique() if n_rows else 0

        if (n_spots > best["n_spots"]) or (n_spots == best["n_spots"] and n_rows > best["n_rows"]):
            best = {"grid_crs": gcrs, "spot_crs": scrs, "joined": joined, "n_rows": n_rows, "n_spots": n_spots}

print("\n✅ BEST COMBO")
print("grid_crs:", best["grid_crs"])
print("spot_crs:", best["spot_crs"])
print("joined rows:", best["n_rows"])
print("unique spots mapped:", best["n_spots"])

if best["n_spots"] < 20:
    raise RuntimeError(
        "매핑된 spot이 너무 적어(20 미만). 격자 파일이 성수 격자가 아니거나, 좌표가 다른 체계일 가능성이 커.\n"
        "grid bounds/spot range 출력값 캡쳐해서 보여주면 바로 맞춰줄 수 있어."
    )

# raw -> spot-day wide(3칼럼)
raw = pd.read_csv(RAW_MULTI)
raw.columns = [c.lower() for c in raw.columns]
vol_candidates = [c for c in raw.columns if c in ["vol","volume","traffic","trfvlm","traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼 못 찾음. raw cols={list(raw.columns)}")
VOL_COL = vol_candidates[0]

raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)

spot_day_hh = raw.groupby(["spot","yyyymmdd","hh"], as_index=False)[VOL_COL].sum()
wide = spot_day_hh.pivot(index=["spot","yyyymmdd"], columns="hh", values=VOL_COL).fillna(0).reset_index()

rename_map = {"01":"traffic_01_02","02":"traffic_02_03","03":"traffic_03_04"}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh:newc}, inplace=True)
    else:
        wide[newc] = 0

wide = wide[["spot","yyyymmdd","traffic_01_02","traffic_02_03","traffic_03_04"]].copy()

# spot->grid 매핑 (best 조합 사용)
joined = best["joined"]
spot_to_grid = joined[[SPOT_COL, "grid_id"]].drop_duplicates().copy()
spot_to_grid.rename(columns={SPOT_COL:"spot"}, inplace=True)
spot_to_grid["spot"] = spot_to_grid["spot"].astype(str)

tmp = wide.merge(spot_to_grid, on="spot", how="inner")
grid_day = tmp.groupby(["grid_id","yyyymmdd"], as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum()
grid_med = grid_day.groupby("grid_id", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]].median()

grid_med.to_csv(OUT_TRAFFIC, index=False, encoding="utf-8-sig")

print("\n✅ saved:", OUT_TRAFFIC)
print("grid_med rows:", len(grid_med))
print("nonzero grids:", (grid_med[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum(axis=1) > 0).sum())
grid_med.head()


grid.crs(original): EPSG:4326
grid bounds: [127.02822996  37.52908006 127.06787684  37.55387744]
spot x/y range: (np.float64(181967.262021), np.float64(215469.587229)) (np.float64(437292.624732), np.float64(465841.155869))

✅ BEST COMBO
grid_crs: EPSG:4326
spot_crs: EPSG:5181
joined rows: 2
unique spots mapped: 2


RuntimeError: 매핑된 spot이 너무 적어(20 미만). 격자 파일이 성수 격자가 아니거나, 좌표가 다른 체계일 가능성이 커.
grid bounds/spot range 출력값 캡쳐해서 보여주면 바로 맞춰줄 수 있어.

In [23]:
import pandas as pd
import numpy as np
import geopandas as gpd

PRO_DIR = ROOT / "data" / "processed"
RAW_DIR = ROOT / "data" / "raw" / "traffic"

GRID_PATH = PRO_DIR / "seongsu_grid_250m_enriched.geojson"
RAW_MULTI = RAW_DIR / "volinfo_hourly_raw_20251201_1214.csv"
SPOTINFO  = RAW_DIR / "spotinfo.csv"

OUT_TRAFFIC = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214_NEAREST.csv"
OUT_GEOJSON = PRO_DIR / "seongsu_grid_250m_plus_traffic_3cols_median_20251201_1214_NEAREST.geojson"

# 1) grid (4326)
grid = gpd.read_file(GRID_PATH)
grid.columns = [c.lower() for c in grid.columns]
if grid.crs is None:
    grid = grid.set_crs("EPSG:4326", allow_override=True)
GRID_CRS = grid.crs.to_string()
assert GRID_CRS.upper() == "EPSG:4326"

# 2) raw -> spot-day wide -> spot별 14일 median (3칼럼)
raw = pd.read_csv(RAW_MULTI)
raw.columns = [c.lower() for c in raw.columns]

vol_candidates = [c for c in raw.columns if c in ["vol","volume","traffic","trfvlm","traffic_volume"]]
if not vol_candidates:
    raise KeyError(f"교통량 값 컬럼 못 찾음. raw cols={list(raw.columns)}")
VOL_COL = vol_candidates[0]

raw[VOL_COL] = pd.to_numeric(raw[VOL_COL], errors="coerce").fillna(0)
raw["spot"] = raw["spot"].astype(str)
raw["hh"] = raw["hh"].astype(str).str.zfill(2)

spot_day_hh = raw.groupby(["spot","yyyymmdd","hh"], as_index=False)[VOL_COL].sum()
wide = spot_day_hh.pivot(index=["spot","yyyymmdd"], columns="hh", values=VOL_COL).fillna(0).reset_index()

rename_map = {"01":"traffic_01_02","02":"traffic_02_03","03":"traffic_03_04"}
for hh, newc in rename_map.items():
    if hh in wide.columns:
        wide.rename(columns={hh:newc}, inplace=True)
    else:
        wide[newc] = 0

wide = wide[["spot","yyyymmdd","traffic_01_02","traffic_02_03","traffic_03_04"]].copy()

spot_med = (
    wide.groupby("spot", as_index=False)[["traffic_01_02","traffic_02_03","traffic_03_04"]]
        .median()
)

print("✅ spot_med rows:", len(spot_med))
print("✅ spot_med nonzero:", (spot_med[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum(axis=1) > 0).sum())

# 3) SpotInfo(TM) -> geometry 만들기 (spot CRS는 우선 EPSG:5186로 가정)
spots = pd.read_csv(SPOTINFO)
spots.columns = [c.lower() for c in spots.columns]

SPOT_COL = "spot_num"
X_COL = "grs80tm_x"
Y_COL = "grs80tm_y"

spots[SPOT_COL] = spots[SPOT_COL].astype(str)
spots[X_COL] = pd.to_numeric(spots[X_COL], errors="coerce")
spots[Y_COL] = pd.to_numeric(spots[Y_COL], errors="coerce")
spots = spots.dropna(subset=[X_COL, Y_COL]).copy()

g_spots = gpd.GeoDataFrame(
    spots[[SPOT_COL, X_COL, Y_COL]].copy(),
    geometry=gpd.points_from_xy(spots[X_COL], spots[Y_COL]),
    crs="EPSG:5181"   # ← 숫자대(18만/43만)가 여기랑 잘 맞음
).to_crs("EPSG:5181")

# spot_med 붙이기
g_spots = g_spots.rename(columns={SPOT_COL:"spot"})
g_spots["spot"] = g_spots["spot"].astype(str)
g_spots = g_spots.merge(spot_med, on="spot", how="left")
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    g_spots[c] = g_spots[c].fillna(0)

# 4) grid centroid 만들고 (거리 계산은 미터계가 좋아서 5186으로 변환)
g_grid_m = grid.to_crs("EPSG:5181").copy()
cent = g_grid_m.copy()
cent["geometry"] = g_grid_m.geometry.centroid

# 5) centroid -> 가장 가까운 spot 붙이기
# max_distance는 넉넉하게(예: 5000m). 너무 멀면 None 나올 수 있어서 0으로 채움.
joined = gpd.sjoin_nearest(
    cent[["grid_id","geometry"]],
    g_spots[["spot","traffic_01_02","traffic_02_03","traffic_03_04","geometry"]],
    how="left",
    distance_col="dist_m",
)

grid_traffic = joined[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04","dist_m"]].copy()
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    grid_traffic[c] = grid_traffic[c].fillna(0)

grid_traffic.to_csv(OUT_TRAFFIC, index=False, encoding="utf-8-sig")
print("✅ saved traffic:", OUT_TRAFFIC, "| rows:", len(grid_traffic))

# 6) geojson에도 붙여서 저장(원래 4326 grid에 join)
grid2 = grid.merge(grid_traffic.drop(columns=["dist_m"]), on="grid_id", how="left")
for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    grid2[c] = grid2[c].fillna(0)

grid2.to_file(OUT_GEOJSON, driver="GeoJSON", encoding="utf-8")
print("✅ saved geojson:", OUT_GEOJSON)

grid2[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]].head(10)


✅ spot_med rows: 137
✅ spot_med nonzero: 136
✅ saved traffic: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\processed\seongsu_grid_traffic_3cols_median_20251201_1214_NEAREST.csv | rows: 109
✅ saved geojson: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\processed\seongsu_grid_250m_plus_traffic_3cols_median_20251201_1214_NEAREST.geojson


,grid_id,traffic_01_02,traffic_02_03,traffic_03_04
0,0,2808.0,2278.0,1958.0
1,1,2808.0,2278.0,1958.0
2,2,752.5,589.0,449.5
3,3,752.5,589.0,449.5
4,4,2808.0,2278.0,1958.0
5,5,2808.0,2278.0,1958.0
6,6,2808.0,2278.0,1958.0
7,7,2808.0,2278.0,1958.0
8,8,2808.0,2278.0,1958.0
9,9,752.5,589.0,449.5


In [24]:
import pandas as pd
from pathlib import Path

PRO_DIR = ROOT / "data" / "processed"

BASE_FEATURES = PRO_DIR / "grid_features_base.csv"
TRAFFIC_3COLS = PRO_DIR / "seongsu_grid_traffic_3cols_median_20251201_1214_NEAREST.csv"
OUT_FINAL     = PRO_DIR / "grid_features_base_plus_traffic3cols_20251201_1214_NEAREST.csv"

base = pd.read_csv(BASE_FEATURES, dtype={"grid_id": str})
traffic = pd.read_csv(TRAFFIC_3COLS, dtype={"grid_id": str})

# grid_id 컬럼명 통일
base.columns = [c if c.lower() != "grid_id" else "grid_id" for c in base.columns]
traffic.columns = [c if c.lower() != "grid_id" else "grid_id" for c in traffic.columns]

merged = base.merge(
    traffic[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]],
    on="grid_id",
    how="left"
)

for c in ["traffic_01_02","traffic_02_03","traffic_03_04"]:
    merged[c] = merged[c].fillna(0)

merged.to_csv(OUT_FINAL, index=False, encoding="utf-8-sig")

print("✅ base rows:", len(base))
print("✅ merged rows:", len(merged))
print("✅ nonzero grids:", (merged[["traffic_01_02","traffic_02_03","traffic_03_04"]].sum(axis=1) > 0).sum(), "/", len(merged))
print("✅ saved:", OUT_FINAL)

merged[["grid_id","traffic_01_02","traffic_02_03","traffic_03_04"]].head(10)


✅ base rows: 109
✅ merged rows: 109
✅ nonzero grids: 109 / 109
✅ saved: C:\Users\A\OneDrive\바탕 화면\seoul-dimming-system\data\processed\grid_features_base_plus_traffic3cols_20251201_1214_NEAREST.csv


,grid_id,traffic_01_02,traffic_02_03,traffic_03_04
0,0,2808.0,2278.0,1958.0
1,1,2808.0,2278.0,1958.0
2,2,752.5,589.0,449.5
3,3,752.5,589.0,449.5
4,4,2808.0,2278.0,1958.0
5,5,2808.0,2278.0,1958.0
6,6,2808.0,2278.0,1958.0
7,7,2808.0,2278.0,1958.0
8,8,2808.0,2278.0,1958.0
9,9,752.5,589.0,449.5
